In [5]:
import torch
import torch.nn as nn
import argparse
import os
import time
import auraloss
import soundfile as sf
import matplotlib.pyplot as plt
from collections import OrderedDict
from flamo.processor import dsp, system
from flamo.optimize.loss import sparsity_loss
from flamo.utils import save_audio
from flamo.functional import signal_gallery, find_onset
from flamo.auxiliary.reverb import parallelFDNAccurateGEQ

torch.manual_seed(130799)


"""
Example function that demonstrates the construction and training of a Feedback Delay Network (FDN) model.
Args:
    args: A dictionary or object containing the necessary arguments for the function.
Returns:
    None
"""

args={
    "nfft": 44100,
    "device": "cuda" if torch.cuda.is_available() else "cpu",
    "samplerate": 44100,
}

args= argparse.Namespace(**args)

# FDN parameters
N = 6  # number of delays
alias_decay_db = 30  # alias decay in dB
delay_lengths = torch.tensor([593, 743, 929, 1153, 1399, 1699])

## ---------------- CONSTRUCT FDN ---------------- ##

# Input and output gains
input_gain = dsp.Gain(
    size=(N, 2),
    nfft=args.nfft,
    requires_grad=True,
    alias_decay_db=alias_decay_db,
    device=args.device,
)
output_gain = dsp.Gain(
    size=(2, N),
    nfft=args.nfft,
    requires_grad=True,
    alias_decay_db=alias_decay_db,
    device=args.device,
)
# Feedback loop with delays
delays = dsp.parallelDelay(
    size=(N,),
    max_len=delay_lengths.max(),
    nfft=args.nfft,
    isint=True,
    requires_grad=False,
    alias_decay_db=alias_decay_db,
    device=args.device,
)
delays.assign_value(delays.sample2s(delay_lengths))
# Feedback path with orthogonal matrix
mixing_matrix = dsp.Matrix(
    size=(N, N),
    nfft=args.nfft,
    matrix_type="orthogonal",
    requires_grad=True,
    alias_decay_db=alias_decay_db,
    device=args.device,
)
attenuation = dsp.parallelGEQ(
    size=(N,),
    octave_interval=1,
    nfft=args.nfft,
    fs=args.samplerate,
    requires_grad=True,
    alias_decay_db=alias_decay_db,
    device=args.device,
)
attenuation.map = lambda x: 20 * torch.log10(torch.sigmoid(x))
feedback = system.Series(
    OrderedDict({"mixing_matrix": mixing_matrix, "attenuation": attenuation})
)

# Recursion
feedback_loop = system.Recursion(fF=delays, fB=feedback)

# Full FDN
FDN = system.Series(
    OrderedDict(
        {
            "input_gain": input_gain,
            "feedback_loop": feedback_loop,
            "output_gain": output_gain,
        }
    )
)

# Create the model with Shell
input_layer = dsp.FFT(args.nfft)
# Since time aliasing mitigation is enabled, we use the iFFTAntiAlias layer
# to undo the effect of the anti aliasing modulation introduced by the system's layers
output_layer = dsp.iFFTAntiAlias(
    nfft=args.nfft, alias_decay_db=alias_decay_db, device=args.device
)
model = system.Shell(core=FDN, input_layer=input_layer, output_layer=output_layer)

ir=model.get_time_response()




In [6]:
print(ir.shape)

torch.Size([1, 44100, 2])


In [3]:
import plotly.express as px
px.line(
    x=torch.arange(ir.shape[-2]) / args.samplerate,
    y=ir[0, :,0].cpu().numpy(),
    labels={"x": "Time (s)", "y": "Amplitude"},
    title="Impulse Response of FDN",
).show()

In [11]:


import torch
import os

import sys
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '..')))

device=torch.device("cuda" if torch.cuda.is_available() else "cpu")

file_id=6448

base_path="/home/eloi/projects/project_mfm_eloi/audio_examples/TM_val/"
file_dry=os.path.join(base_path, "dry",str(file_id),"vocals.wav")
file_wet=os.path.join(base_path, "wet",str(file_id),"vocals.wav")

file_dry_fxnorm=os.path.join(base_path,"dry", str(file_id), "vocals_normalized.wav")
file_wet_fxnorm=os.path.join(base_path, "wet",str(file_id), "vocals_normalized.wav")

file_dry_fxnorm_dr=os.path.join(base_path, "dry", str(file_id), "vocals_normalized_dr.wav")
file_wet_fxnorm_dr=os.path.join(base_path, "wet",str(file_id),"vocals_normalized_dr.wav")

import soundfile as sf

dry, sr = sf.read(file_dry)
wet, sr = sf.read(file_wet)

dry=torch.from_numpy(dry.T).float().unsqueeze(0)
wet=torch.from_numpy(wet.T).float().unsqueeze(0)

dry=dry.mean(dim=1, keepdim=True)



start_t=15*sr
segment_length = 524288
print("dry", dry.shape, "wet", wet.shape)
dry_segment = dry[...,start_t:start_t + segment_length]
wet_segment = wet[...,start_t:start_t + segment_length]

print("dry_segment", dry_segment.shape)
print("wet_segment", wet_segment.shape)

#import pyloudnorm as pyln
#meter = pyln.Meter(sr)
#normaliser = lambda x: pyln.normalize.loudness(
#    x, meter.integrated_loudness(x), -18.0
#)

#print("dry", dry.shape, dry.min(), dry.max())
#
#dry = torch.from_numpy(normaliser(dry.numpy().T).T).float().to(device)
#wet = torch.from_numpy(normaliser(wet.numpy().T).T).float().to(device)



#print("dry", dry.shape,dry.min(), dry.max())



dry torch.Size([1, 1, 11065479]) wet torch.Size([1, 2, 11065479])
dry_segment torch.Size([1, 1, 524288])
wet_segment torch.Size([1, 2, 524288])


In [12]:
y=model(dry_segment.to(device))

OutOfMemoryError: CUDA out of memory. Tried to allocate 187.50 GiB. GPU 0 has a total capacity of 79.15 GiB of which 68.30 GiB is free. Process 2868585 has 10.31 GiB memory in use. Including non-PyTorch memory, this process has 526.00 MiB memory in use. Of the allocated memory 16.75 MiB is allocated by PyTorch, and 19.25 MiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)